In [ ]:
import pandas as pd
from transformers import pipeline
import json

Load the .csv file created and uploaded by https://github.com/matthewgthomas/gratitude/blob/main/nlp/convert%20gratitudes%20to%20table.R

In [ ]:
grats = pd.read_csv("drive/MyDrive/Gratitude/gratitudes.csv")

In [ ]:
grats.head()

Load the [deberta zero-shot classifier](https://huggingface.co/MoritzLaurer/deberta-v3-large-zeroshot-v2.0)

In [ ]:
# hypothesis_template = "This text is about {}"
hypothesis_template = "The topic of this text is {}"

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0", device=0)

In [ ]:
gratitude_themes = [
    "socialising",
    "food and cooking",
    "solitude and self-care",
    "nature and travel"
    ]

Classify each gratitude journal entry into one of the themes

In [ ]:
grats_labels = []

for index, row in grats.iterrows():
  text = row['gratitude']
  output = zeroshot_classifier(text, gratitude_themes, hypothesis_template=hypothesis_template, multi_label=True)
  grats_labels.append(output)
  print(f"Finished {index}")

Join the filtered themes to the gratitude journal entries and save

In [ ]:
grats_labels_df = pd.DataFrame(grats_labels)
grats_labels_df.head()

In [ ]:
grats_merged = pd.merge(grats, grats_labels_df, left_on='gratitude', right_on='sequence', how='left')
grats_merged.head()

In [ ]:
grats_merged.to_csv("drive/MyDrive/Gratitude/gratitudes_labeled.csv", index=False)

The grats_labels dict contains a list of labels and a list of scores. Keep only the labels whose corresponding scores are > 0.5

In [ ]:
filtered_grats_labels = []
for item in grats_labels:
  filtered_labels = [label for label, score in zip(item['labels'], item['scores']) if score > 0.5]
  filtered_grats_labels.append({'sequence': item['sequence'], 'labels': filtered_labels})


In [ ]:
filtered_grats_labels

Join the filtered themes to the gratitude journal entries and save

In [ ]:
filtered_grats_labels_df = pd.DataFrame(filtered_grats_labels)
grats_merged = pd.merge(grats, filtered_grats_labels_df, left_on='gratitude', right_on='sequence', how='left')
grats_merged.head()

In [ ]:
grats_merged.to_csv("drive/MyDrive/Gratitude/gratitudes_labeled_filtered.csv", index=False)